In [1]:
!pip3 install pandas
!pip3 install tensorflow
import pandas as pd
import numpy as np
import tensorflow as tf

# Assuming you have a dataset like the one you provided
# "Date" column should be in datetime format
# "Receipt_Count" column represents the number of receipts received on each day

# Load your dataset


df = pd.read_csv('data_daily.csv')
print(df.head(5))
print(df.columns)
print(df.dtypes)
df['# Date'] = pd.to_datetime(df['# Date'])
df['Month'] = df['# Date'].dt.month
df['Year'] = df['# Date'].dt.year

# Feature engineering
df_agg = df.groupby(['Year', 'Month'], as_index=False)['Receipt_Count'].sum()
df_agg['Prev_Month_Receipts'] = df_agg['Receipt_Count'].shift(1)

# Drop the first row as it will have NaN due to shift
df_agg = df_agg.dropna()

# Model data preparation
X = df_agg[['Prev_Month_Receipts']].values
y = df_agg['Receipt_Count'].values

# Normalize data manually
X_mean, X_std = X.mean(), X.std()
X_normalized = (X - X_mean) / X_std

y_mean, y_std = y.mean(), y.std()
y_normalized = (y - y_mean) / y_std

# Split data into training and testing sets
split_ratio = 0.8
split_index = int(len(X_normalized) * split_ratio)

X_train, X_test = X_normalized[:split_index], X_normalized[split_index:]
y_train, y_test = y_normalized[:split_index], y_normalized[split_index:]

# Model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_dim=1, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer, loss='mean_squared_error')

# Training
model.fit(X_train, y_train, epochs=100, batch_size=8, verbose=2)

# Evaluation
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on Test Set: {loss}')

# Inference for the year 2022
# Assuming new_data is the input for each day's receipts in 2021
# predicted_normalized = model.predict(X_normalized).flatten()
# predicted = predicted_normalized * y_std + y_mean

# # Print the predicted number of receipts for each month of 2022
# print(predicted)


# Generate predictions for the next 12 values
future_values = []

# Use the last sequence of known values to predict the next value

last_sequence = X_normalized[-1]
for _ in range(12):
    prediction = model.predict(last_sequence)
    future_values.append(prediction.flatten())
    last_sequence = prediction.flatten()
   

# Denormalize the predictions
future_values = np.array(future_values) * y_std + y_mean

print(future_values)




[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


       # Date  Receipt_Count
0  2021-01-01        7564766
1  2021-01-02        7455524
2  2021-01-03        7095414
3  2021-01-04        7666163
4  2021-01-05        7771289
Index(['# Date', 'Receipt_Count'], dtype='object')
# Date           object
Receipt_Count     int64
dtype: object
Epoch 1/100
1/1 - 0s - loss: 0.9887 - 183ms/epoch - 183ms/step
Epoch 2/100
1/1 - 0s - loss: 0.7651 - 1ms/epoch - 1ms/step
Epoch 3/100
1/1 - 0s - loss: 0.5848 - 1ms/epoch - 1ms/step
Epoch 4/100
1/1 - 0s - loss: 0.4453 - 947us/epoch - 947us/step
Epoch 5/100
1/1 - 0s - loss: 0.3426 - 1ms/epoch - 1ms/step
Epoch 6/100
1/1 - 0s - loss: 0.2746 - 1ms/epoch - 1ms/step
Epoch 7/100
1/1 - 0s - loss: 0.2380 - 2ms/epoch - 2ms/step
Epoch 8/100
1/1 - 0s - loss: 0.2295 - 2ms/epoch - 2ms/step
Epoch 9/100
1/1 - 0s - loss: 0.2422 - 1ms/epoch - 1ms/step
Epoch 10/100
1/1 - 0s - loss: 0.2658 - 2ms/epoch - 2ms/step
Epoch 11/100
1/1 - 0s - loss: 0.2880 - 2ms/epoch - 2ms/step
Epoch 12/100
1/1 - 0s - loss: 0.2995 - 1ms/epoch - 1ms

In [4]:
#method 2

df = pd.read_csv('data_daily.csv')
print(df.head(5))
print(df.columns)
print(df.dtypes)
df['# Date'] = pd.to_datetime(df['# Date'])
df['Month'] = df['# Date'].dt.month
df['Year'] = df['# Date'].dt.year


       # Date  Receipt_Count
0  2021-01-01        7564766
1  2021-01-02        7455524
2  2021-01-03        7095414
3  2021-01-04        7666163
4  2021-01-05        7771289
Index(['# Date', 'Receipt_Count'], dtype='object')
# Date           object
Receipt_Count     int64
dtype: object


In [5]:
print(df.head(5))

      # Date  Receipt_Count  Month  Year
0 2021-01-01        7564766      1  2021
1 2021-01-02        7455524      1  2021
2 2021-01-03        7095414      1  2021
3 2021-01-04        7666163      1  2021
4 2021-01-05        7771289      1  2021


In [6]:
# Feature engineering
df_agg = df.groupby(['Year', 'Month'], as_index=False)['Receipt_Count'].sum()
df_agg['Prev_Month_Receipts'] = df_agg['Receipt_Count'].shift(1)
print(df_agg.head(14))

    Year  Month  Receipt_Count  Prev_Month_Receipts
0   2021      1      236736687                  NaN
1   2021      2      220033460          236736687.0
2   2021      3      248608625          220033460.0
3   2021      4      250644830          248608625.0
4   2021      5      263151748          250644830.0
5   2021      6      260656840          263151748.0
6   2021      7      274776003          260656840.0
7   2021      8      283943231          274776003.0
8   2021      9      281146154          283943231.0
9   2021     10      295965185          281146154.0
10  2021     11      296085162          295965185.0
11  2021     12      309948684          296085162.0


In [7]:
# Drop the first row as it will have NaN due to shift
df_agg = df_agg.dropna()

# Model data preparation
X = df_agg[['Prev_Month_Receipts']].values
y = df_agg['Receipt_Count'].values

# Normalize data manually
X_mean, X_std = X.mean(), X.std()
X_normalized = (X - X_mean) / X_std

y_mean, y_std = y.mean(), y.std()
y_normalized = (y - y_mean) / y_std




In [28]:
# Inference for the year 2022
# Generate predictions for the next 12 values
sequence_length = 1

# Split data into training and testing sets
split_ratio = 0.8
split_index = int(len(X_normalized) * split_ratio)

X_train, X_test = X_normalized[:split_index], X_normalized[split_index:]
y_train, y_test = y_normalized[:split_index], y_normalized[split_index:]

# Reshape for LSTM input
X_normalized = X_normalized.reshape(X_normalized.shape[0], X_normalized.shape[1], 1)

# Build the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(500, input_shape=(sequence_length, 1)),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')



# Training
model.fit(X_train, y_train, epochs=100, batch_size=8, verbose=2)

# Evaluation
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on Test Set: {loss}')

# Generate predictions for the next 12 values
future_values = []

# Use the last sequence of known values to predict the next value
last_sequence = X_normalized[-1].reshape(1, sequence_length, 1)

for _ in range(12):
    prediction = model.predict(last_sequence)
    future_values.append(prediction.flatten())
    last_sequence = np.roll(last_sequence, -1, axis=1)
    last_sequence[0, -1, 0] = prediction

# Denormalize the predictions
future_values = np.array(future_values) * y_std + y_mean

print(future_values)

Epoch 1/100
1/1 - 0s - loss: 0.8488 - 445ms/epoch - 445ms/step
Epoch 2/100
1/1 - 0s - loss: 0.8267 - 3ms/epoch - 3ms/step
Epoch 3/100
1/1 - 0s - loss: 0.8051 - 3ms/epoch - 3ms/step
Epoch 4/100
1/1 - 0s - loss: 0.7840 - 3ms/epoch - 3ms/step
Epoch 5/100
1/1 - 0s - loss: 0.7635 - 4ms/epoch - 4ms/step
Epoch 6/100
1/1 - 0s - loss: 0.7434 - 3ms/epoch - 3ms/step
Epoch 7/100
1/1 - 0s - loss: 0.7238 - 3ms/epoch - 3ms/step
Epoch 8/100
1/1 - 0s - loss: 0.7047 - 4ms/epoch - 4ms/step
Epoch 9/100
1/1 - 0s - loss: 0.6859 - 4ms/epoch - 4ms/step
Epoch 10/100
1/1 - 0s - loss: 0.6676 - 3ms/epoch - 3ms/step
Epoch 11/100
1/1 - 0s - loss: 0.6497 - 3ms/epoch - 3ms/step
Epoch 12/100
1/1 - 0s - loss: 0.6322 - 4ms/epoch - 4ms/step
Epoch 13/100
1/1 - 0s - loss: 0.6150 - 3ms/epoch - 3ms/step
Epoch 14/100
1/1 - 0s - loss: 0.5982 - 3ms/epoch - 3ms/step
Epoch 15/100
1/1 - 0s - loss: 0.5818 - 3ms/epoch - 3ms/step
Epoch 16/100
1/1 - 0s - loss: 0.5657 - 4ms/epoch - 4ms/step
Epoch 17/100
1/1 - 0s - loss: 0.5500 - 3ms/ep

/var/folders/6w/1czj28vj3_q0hl9l50q3p0m80000gn/T/ipykernel_79330/3741129475.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  last_sequence[0, -1, 0] = prediction


1/1 [==============================] - 0s 8ms/step
[[2.9216653e+08]
 [2.8343405e+08]
 [2.7720746e+08]
 [2.7269818e+08]
 [2.6939962e+08]
 [2.6697045e+08]
 [2.6517322e+08]
 [2.6383920e+08]
 [2.6284674e+08]
 [2.6210714e+08]
 [2.6155531e+08]
 [2.6114322e+08]]


In [42]:
df_agg['Receipt_Count']



1     220033460
2     248608625
3     250644830
4     263151748
5     260656840
6     274776003
7     283943231
8     281146154
9     295965185
10    296085162
11    309948684
Name: Receipt_Count, dtype: int64

In [43]:
#method 3: using difference in values

In [91]:
import numpy as np
import pandas as pd
import tensorflow as tf


# Provided data
df = pd.read_csv('data_daily.csv')
print(df.head(5))
print(df.columns)
print(df.dtypes)
df['# Date'] = pd.to_datetime(df['# Date'])
df['Month'] = df['# Date'].dt.month
df['Year'] = df['# Date'].dt.year

# Feature engineering
df_agg = df.groupby(['Year', 'Month'], as_index=False)['Receipt_Count'].sum()


# Take the difference between consecutive values
df_agg['Diff_Values'] = df_agg['Receipt_Count'].diff().fillna(0)

# Create input and target data for training
X_train = df_agg['Diff_Values'][:-1].values.reshape(-1, 1, 1)
y_train = df_agg['Diff_Values'][1:].values.reshape(-1, 1)

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_dim=1, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer, loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=8, verbose=2)

# Generate predictions for the next 12 values
X_pred = df_agg['Diff_Values'].values[-1].reshape(1, 1, 1)
y_pred_diff = []
for _ in range(12):
    pred_diff = model.predict(X_pred)
    y_pred_diff.append(pred_diff.flatten())
    X_pred = pred_diff.reshape(1, 1, 1)

print("1:",df_agg['Diff_Values'].values)
print("2:", y_pred_diff)
res = np.concatenate(y_pred_diff)
print("3:", res)
# Cumulatively sum the differences to obtain predictions for the original time series
y_pred_cumsum = np.cumsum(np.concatenate([df_agg['Diff_Values'].values, res]))
                                          
# print(y_pred_cumsum)
                                          
                                          

       # Date  Receipt_Count
0  2021-01-01        7564766
1  2021-01-02        7455524
2  2021-01-03        7095414
3  2021-01-04        7666163
4  2021-01-05        7771289
Index(['# Date', 'Receipt_Count'], dtype='object')
# Date           object
Receipt_Count     int64
dtype: object
Epoch 1/100
2/2 - 0s - loss: 190418968903680.0000 - 136ms/epoch - 68ms/step
Epoch 2/100
2/2 - 0s - loss: 176104782430208.0000 - 2ms/epoch - 820us/step
Epoch 3/100
2/2 - 0s - loss: 162858532864000.0000 - 2ms/epoch - 818us/step
Epoch 4/100
2/2 - 0s - loss: 155980931268608.0000 - 1ms/epoch - 745us/step
Epoch 5/100
2/2 - 0s - loss: 145096192294912.0000 - 1ms/epoch - 646us/step
Epoch 6/100
2/2 - 0s - loss: 135541643280384.0000 - 1ms/epoch - 641us/step
Epoch 7/100
2/2 - 0s - loss: 124633701416960.0000 - 2ms/epoch - 1ms/step
Epoch 8/100
2/2 - 0s - loss: 119472165748736.0000 - 2ms/epoch - 962us/step
Epoch 9/100
2/2 - 0s - loss: 112909891928064.0000 - 3ms/epoch - 1ms/step
Epoch 10/100
2/2 - 0s - loss: 10599432349

1/1 [==============================] - 0s 7ms/step
1: [        0. -16703227.  28575165.   2036205.  12506918.  -2494908.
  14119163.   9167228.  -2797077.  14819031.    119977.  13863522.]
2: [array([906203.1], dtype=float32), array([59238.965], dtype=float32), array([3876.5488], dtype=float32), array([257.74548], dtype=float32), array([21.199844], dtype=float32), array([5.66588], dtype=float32), array([3.7440014], dtype=float32), array([2.9484367], dtype=float32), array([2.6191115], dtype=float32), array([2.4827871], dtype=float32), array([2.4263554], dtype=float32), array([2.4029956], dtype=float32)]
3: [9.0620312e+05 5.9238965e+04 3.8765488e+03 2.5774548e+02 2.1199844e+01
 5.6658802e+00 3.7440014e+00 2.9484367e+00 2.6191115e+00 2.4827871e+00
 2.4263554e+00 2.4029956e+00]


In [92]:
predictions = []
dec2021_val = 309948684
i = 1
j = 1
fir = 309948684 + res[0]
predictions.append(fir)
while(i<12 and j<12):
    predictions.append(predictions[i-1] + y_pred_cumsum[j])
    i+=1
    j+=1
print(predictions)
    
    

[310854887.125, 294151660.125, 306023598.125, 319931741.125, 346346802.125, 370266955.125, 408306271.125, 455512815.125, 499922282.125, 559150780.125, 618499255.125, 691711252.125]


In [85]:
a = 0.7
b = 0.3
final = []
for i in range(12):
    final.append(a * future_values[i][0] + b * predictions[i])
print(final)

[286249098.525, 281891807.22499996, 286336867.025, 291703364.325, 301241668.225, 309576600.525, 321191652.92499995, 335384752.125, 348710123.42499995, 366478807.225, 384283372.125, 406246971.225]
